## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
%run ./includes/includes


In [0]:
%run ./includes/utilities

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
# Setting the number of partitions to be 2 times the number of cores available to us
spark.conf.set("spark.sql.shuffle.partitions", str(2 * sc.defaultParallelism))

from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType
from pyspark.sql.functions import from_json, input_file_name, current_timestamp, lit,when, count, col, desc
import matplotlib.pyplot as plt
from pyspark.sql.functions import split, size, expr, concat_ws, array, lit, regexp_replace, to_timestamp,explode, min, max
import mlflow
from mlflow import pyfunc
from mlflow.models import Model
import pandas as pd
from pyspark.sql.functions import pandas_udf, PandasUDFType, sum
import mlflow.sklearn
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import numpy as np
import time
inactivity_threshold = 10000

## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
# ENTER YOUR CODE HERE
source_files = get_source_listing_df()
print(f"The number of source files are: {source_files.shape[0]}")
print(f"The content stored in the first json file is as follows: ")
print(show_s3_file_contents(source_files['File Name'][0]))

## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
# Ensuring idempotency
stop_all_streams() 
clear_previous_run()

In [0]:
BRONZE_CHECKPOINT = "/tmp/labuser104917-3007333/bronze.checkpoint"
BRONZE_DELTA = "/tmp/labuser104917-3007333/bronze.delta"

# Making schema
schema = StructType([
    StructField("date", StringType(), True),
    StructField("user", StringType(), True),
    StructField("text", StringType(), True),
    StructField("sentiment", StringType(), True),
    StructField("source_file", StringType(), True),
    StructField("processing_time", TimestampType(), True)
])

# We will be using the following directory that contains all the source files given in the desciprtion table above    
source_file_folder = "s3a://voc-75-databricks-data/voc_volume/"

# Create the streaming DataFrame
df = (spark.readStream
      .format("cloudFiles")
      .option("cloudFiles.format", "json")
      .option("maxFilesPerTrigger", 1)
      .schema(schema)
      .load(source_file_folder)
      .withColumn("source_file", input_file_name())
      .withColumn("processing_time", current_timestamp()))

query = (df.writeStream
         .format("delta")
         .outputMode('append')
         .option("checkpointLocation", BRONZE_CHECKPOINT)
         .queryName("bronze_stream")
         .option("mergeSchema", "true")
         .start(BRONZE_DELTA))

# Wait for the query to terminate
# query.awaitTermination()

In [0]:
query.awaitTermination()


##4.0 Bronze Data Exploratory Data Analysis
* How many tweets are captured in your Bronze Table?
* Are there any columns that contain Nan or Null values? If so how many and what will you do in your silver transforms to address this?
* Count the number of tweets by each unique user handle and sort the data by descending count.
* How many tweets have at least one mention (@) how many tweet have no mentions (@)
* Plot a bar chart that shows the top 20 tweeters (users)

In [0]:
spark.readStream.load(BRONZE_DELTA).createOrReplaceTempView('bronze_table')

In [0]:
%sql
-- The number of tweets captured in the Bronze Table
select count(*)
from bronze_table

In [0]:
%sql
-- Seeing the null values present in each column of the bronze table
SELECT
    SUM(CASE WHEN date IS NULL THEN 1 ELSE 0 END) AS date_nulls,
    SUM(CASE WHEN user IS NULL THEN 1 ELSE 0 END) AS user_nulls,
    SUM(CASE WHEN text IS NULL THEN 1 ELSE 0 END) AS text_nulls,
    SUM(CASE WHEN sentiment IS NULL THEN 1 ELSE 0 END) AS sentiment_nulls,
    SUM(CASE WHEN source_file IS NULL THEN 1 ELSE 0 END) AS source_file_nulls,
    SUM(CASE WHEN processing_time IS NULL THEN 1 ELSE 0 END) AS processing_time_nulls
FROM
    bronze_table;

In [0]:
%sql
-- Looking at the top tweeters in descending order
SELECT
    user,
    COUNT(*) AS tweet_counts
FROM
    bronze_table
GROUP BY
    user
ORDER BY
    tweet_counts DESC;

In [0]:
%sql
-- Counting the number of tweets that have at least one mention
select count(*) as tweets_with_mentions
from bronze_table
where text like '%@%'

In [0]:
%sql
-- Counting the number of tweets with no mentions
select count(*) as tweets_with_no_mentions
from bronze_table
where text not like '%@%'

In [0]:
%sql
-- Looking at the top tweeters in descending order limit to top 20 users only
SELECT
    user,
    COUNT(*) AS count
FROM
    bronze_table
GROUP BY
    user
ORDER BY
    count DESC
    LIMIT 20;

Databricks visualization. Run in Databricks to view.

## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
# Setting the legacy time parser policy to avoid errors 
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

SILVER_CHECKPOINT = "/tmp/labuser104917-3007333/silver.checkpoint"
SILVER_DELTA = "/tmp/labuser104917-3007333/silver.delta"

# Read stream from the bronze Delta table
bronze_df = (spark.readStream
    .format("delta")
    .load(BRONZE_DELTA)
    # .load("/tmp/labuser104917-3007333/bronze.delta")
)
    
    

# Split the column by spaces - this operation returns a new column with arrays
bronze_df = bronze_df.withColumn("split_text", split(col("text"), ' '))

# Filter rows where split value length > 1 and the value starts with '@'
filtered = bronze_df.filter((size(col("split_text")) > 1) & expr("exists(split_text, value -> value like '@%'and length(value) > 2)"))

# Transform the DataFrame
silver_df_raw = filtered.select(
    to_timestamp(col("date"), "EEE MMM dd HH:mm:ss z yyyy").alias("timestamp"),
    col("sentiment"),
    explode(expr("FILTER(split_text, word -> startswith(word, '@'))")).alias("mention"),
    expr("ARRAY_JOIN(FILTER(split_text, word -> NOT startswith(word, '@')), ' ')").alias("cleaned_text")
)

# Write stream to the silver Delta table
silver_stream = (silver_df_raw.writeStream
    .format("delta")
    .option("checkpointLocation", SILVER_CHECKPOINT)
    .outputMode('append')
    .queryName("silver_stream")
    .option("mergeSchema", "true")
    .start(SILVER_DELTA)
)

In [0]:
silver_stream.awaitTermination()

## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
# Loading the model first 
model_uri = "models:/HF_TWEET_SENTIMENT/Production"
dependencies = mlflow.pyfunc.get_model_dependencies(model_uri)

In [0]:
sentiment_analysis_udf = mlflow.pyfunc.spark_udf(spark, "models:/HF_TWEET_SENTIMENT/Production")

In [0]:
gold_stream_raw = (spark.readStream
                   .format("delta")
                   .load(SILVER_DELTA))

GOLD_CHECKPOINT = "/tmp/labuser104917-3007333/gold.checkpoint"
GOLD_DELTA	= "/tmp/labuser104917-3007333/gold.delta"


sentiment_df = gold_stream_raw.withColumn("predictions", sentiment_analysis_udf(col("cleaned_text"))) \
    .select(
        col("timestamp"),
        col("mention"),
        col("cleaned_text"),
        col("sentiment"),
        col("predictions.label").alias("predicted_sentiment"),
        col("predictions.score").alias("predicted_score"),
        when(col("sentiment") == "negative", 0).
        when(col("sentiment") == "positive", 1).otherwise(-1).alias("sentiment_id"),
        when(col("predictions.label") == "NEG", 0)
        .when(col("predictions.label") == "POS", 1).otherwise(-1)
        .alias("predicted_sentiment_id")
    )\
    .filter(col("mention") != "@")

gold_stream = (sentiment_df.writeStream
               .format("delta")
               .option("checkpointLocation", GOLD_CHECKPOINT)
               .outputMode("append")
               .queryName("gold_stream")
               .option("mergeSchema", "true")
               .start(GOLD_DELTA))



In [0]:
gold_stream.awaitTermination()    

## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
spark.readStream.load(GOLD_DELTA).createOrReplaceTempView('gold_table')


In [0]:
%sql
-- Count the total number of mentions
SELECT COUNT(*) AS total_mentions
FROM gold_table;


In [0]:
%sql
-- Count the number of neutral, positive, and negative tweets for each mention
SELECT mention, 
       SUM(CASE WHEN predicted_sentiment = 'NEU' THEN 1 ELSE 0 END) AS neutral_count,
       SUM(CASE WHEN predicted_sentiment = 'POS' THEN 1 ELSE 0 END) AS positive_count,
       SUM(CASE WHEN predicted_sentiment = 'NEG' THEN 1 ELSE 0 END) AS negative_count,
       COUNT(*) AS total_count
FROM gold_table
GROUP BY mention
ORDER BY total_count DESC;

In [0]:
%sql
SELECT mention, positive_count
FROM (
  SELECT mention, 
         SUM(CASE WHEN predicted_sentiment = 'POS' THEN 1 ELSE 0 END) AS positive_count
  FROM gold_table
  GROUP BY mention
  ORDER BY positive_count DESC
  LIMIT 20
)

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT mention, negative_count
FROM (
  SELECT mention, 
         SUM(CASE WHEN predicted_sentiment = 'NEG' THEN 1 ELSE 0 END) AS negative_count
  FROM gold_table
  GROUP BY mention
  ORDER BY negative_count DESC
  LIMIT 20
)

Databricks visualization. Run in Databricks to view.

## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
gold_table = spark.read.format("delta").load(GOLD_DELTA)

In [0]:
with mlflow.start_run():
    # Log the model name and MLflow version as parameters
    mlflow.log_param("model_name", "HF_TWEET_SENTIMENT")
    mlflow.log_param("mlflow_version", 1)
    
    # Log the Delta Table version as a parameter
    mlflow.log_param("gold_delta", "1")
    
    # Extract the ground truth and predicted labels
    y_true = gold_table.select("sentiment_id").toPandas()
    y_pred = gold_table.select("predicted_sentiment_id").toPandas()

    # Logging the silver table version
    delta_table = DeltaTable.forPath(spark, SILVER_DELTA)
    latest_silver_version = delta_table.history().select("version").limit(1).collect()[0][0]

    mlflow.log_param("Silver Table Version", str(latest_silver_version))
    # Convert the DataFrames to lists
    y_true = y_true.squeeze().tolist()
    y_pred = y_pred.squeeze().tolist()
    
    # # Compute accuracy
    # accuracy = accuracy_score(y_true, y_pred)
    
    # # Log the accuracy as an MLflow metric
    # mlflow.log_metric("accuracy", accuracy)
    
    # Create a confusion matrix plot
    cm = confusion_matrix(y_true, y_pred)
    
    # Plotting the confusion matrix
    plt.figure(figsize=(6, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    
    # Add cell values to the plot
    for i in range(len(cm)):
        for j in range(len(cm[0])):
            plt.text(j, i, cm[i][j], horizontalalignment='center', color='red')
    
    # Set the tick marks and labels
    tick_marks = np.arange(len(('Neutral', 'Negative', 'Positive')))
    plt.xticks(tick_marks, ('Neutral', 'Negative', 'Positive'), rotation=45)
    plt.yticks(tick_marks, ('Neutral', 'Negative', 'Positive'))
    

    # Log the confusion matrix plot as an MLflow artifact
    plt.savefig("confusion_matrix.png")
    mlflow.log_artifact("confusion_matrix.png")

## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
# ENTER YOUR CODE HERE
stop_all_streams()

In [0]:
# Get the notebooks ending time note START_TIME was established in the include file when the notebook started.
END_TIME = time.time()

In [0]:
print(END_TIME- START_TIME)

## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### Answers

- Regarding the potential spill issue, in UI I didn't observe any data spilling from the pipeline's memory. That means the partition size wasn't large enough to cause a memory overflow.

No Spill: https://drive.google.com/file/d/1_oZWHy8bl6Uvgq0YxbdBlsLBhh4V98EO/view?usp=drive_link

- As for data skew, I didn't notice any significant imbalance in partition sizes. 

- However, since our pipeline includes various wide transformations, such as groupBy, aggregate, distinct, we can somewhat optimize it by predefining the number of clusters.

DAG Silver Stream: https://drive.google.com/file/d/1iOnlDPhF8JMwjH5UaZXNUOC45w8zhd5d/view?usp=drive_link


- Apart from using the model_udf for making predictions, we have serialized our code using await termination after every write stream to ensure that our whole code runs after clicking run all.

- Concerning storage efficiency, the Spark UI shows that the current storage format is not optimized, given the large number of relatively small JSON files being read and processed.

- Since the tweets stored in these JSON files are typically small in size, it contributes to the large number of small files problem. To enhance our code's efficiency, we should address this issue by implementing a solution that optimizes the handling of numerous small files.

* Some jobs failed as well 


Jobs which Failed: https://drive.google.com/file/d/1QYnjkED5gbK0Eoc89uk8Z5YyM8LbMGGP/view?usp=drive_link